In [5]:
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
def KNNPCA(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
       	
    # KNN classsifier
    clf = KNeighborsClassifier(n_neighbors=Kv)
    trained_model=clf.fit(X_train,y_train)
    trained_model.fit(X_train,y_train )
    
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
        
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
        	
    # KNN classsifier
    clf = KNeighborsClassifier(n_neighbors=Ka)
    trained_model=clf.fit(X_train,z_train)
    trained_model.fit(X_train,z_train )
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100
            ,f1_score(z_test, z_predict, average='weighted'),
            f1_score(y_test, y_predict, average='weighted')]

############ Happy And Sad########################33
def KNNPCAHS(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)): #Don't Start training unless you have 2 classes.. happens with imbalance
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    # KNN classsifier
    clf = KNeighborsClassifier(n_neighbors=Kv)
    trained_model=clf.fit(X_train,y_train)
    trained_model.fit(X_train,y_train )
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]


############ All Emotions########################33
def KNNPCACombined(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    # KNN classsifier
    clf = KNeighborsClassifier(n_neighbors=Kv)
    trained_model=clf.fit(X_train,y_train)
    trained_model.fit(X_train,y_train )
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]

################ Run Each Method Multiple Times #####################
def Run100(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    F1_ScoreArousal=[]
    F1_ScoreValence=[]
    for i in range (100):
        KNN=KNNPCA(UserName,Kv,Ka,Components,Window,StartTime,EndTime)
        arousal+=[KNN[0]]
        valence+=[KNN[1]]
        F1_ScoreArousal+=[KNN[2]]
        F1_ScoreValence+=[KNN[3]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence),np.mean(F1_ScoreArousal),np.mean(F1_ScoreValence)];
    

    
def RunCombined100(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_ALL=[]
    for i in range (100):
        AllEmotions=KNNPCACombined(UserName,Kv,Ka,Components,Window,StartTime,EndTime)
        Combined+=[AllEmotions[0]]
        F1_ALL+=[AllEmotions[1]]
    print("Array of average All Emotions accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_ALL)];
    
def RunHS100(UserName,Kv=3,Ka=5,Components=10,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_HS=[]
    for i in range (100):
        HS=KNNPCAHS(UserName,Kv,Ka,Components,Window,StartTime,EndTime)
        Combined+=[HS[0]]
        F1_HS+=[HS[1]]
    print("Array of average Happy Sad accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_HS)];

##################Main#########################
if __name__ == "__main__":
    HSAccurecy=0;
    HSF1=0;
    CombinedAccurecy=0;
    CombinedF1=0;
    ValenceAccurecy=0;
    ValenceF1=0;
    ArousalAccurecy=0;
    ArousalF1=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
        ValenceF1+=var2[3]
    for i in range(10,31):
        print("s"+str(i))
        #Happy and Sad
        var =RunHS100("s"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
    print("Accurecy happy sad for first 30 "+str(HSAccurecy/30))
    print("F1 happy sad for first 30 "+str(HSF1/30))
    print("Accurecy Combined sad for first 30 "+str(CombinedAccurecy/30))
    print("F1 Combined sad for first 30 "+str(CombinedF1/30))
    print("Accurecy Valence for first 30 "+str(ValenceAccurecy/30))
    print("F1 Valence for first 30 "+str(ValenceF1/30))
    print("Accurecy Arousal for first 30 "+str(ArousalAccurecy/30))
    print("F1 Arousal for first 30 "+str(ArousalF1/30))


s01
Array of average Happy Sad accurecies for user s01
Counter({40.0: 35, 60.0: 34, 80.0: 17, 20.0: 9, 0.0: 4, 100.0: 1})
Happy Sad
[50.8, 0.4923492063492064]
Array of average All Emotions accurecies for user s01
Counter({25.0: 34, 37.5: 26, 50.0: 22, 12.5: 10, 62.5: 4, 0.0: 3, 75.0: 1})
Combined Emotions
[33.75, 0.2928820346320346]
Arousal 51.375
Counter({50.0: 31, 37.5: 25, 62.5: 21, 75.0: 14, 25.0: 7, 100.0: 1, 12.5: 1})
Valence 48.75
Counter({50.0: 31, 62.5: 22, 37.5: 21, 25.0: 14, 75.0: 7, 87.5: 3, 12.5: 2})
Arousal Valence
[51.375, 48.75, 0.4655480630480631, 0.47499494949494947]
s02
Array of average Happy Sad accurecies for user s02
Counter({80.0: 51, 60.0: 26, 100.0: 19, 40.0: 4})
Happy Sad
[77.0, 0.7218492063492064]
Array of average All Emotions accurecies for user s02
Counter({50.0: 32, 37.5: 24, 62.5: 19, 25.0: 11, 75.0: 7, 12.5: 4, 87.5: 2, 0.0: 1})
Combined Emotions
[47.125, 0.3991644258519259]
Arousal 64.25
Counter({62.5: 29, 75.0: 23, 50.0: 21, 87.5: 12, 37.5: 9, 100.0: 4

Array of average Happy Sad accurecies for user s15
Counter({50.0: 36, 75.0: 32, 25.0: 26, 0.0: 4, 100.0: 2})
Happy Sad
[50.5, 0.4809047619047619]
Array of average All Emotions accurecies for user s15
Counter({37.5: 41, 25.0: 23, 12.5: 13, 50.0: 12, 0.0: 7, 62.5: 3, 75.0: 1})
Combined Emotions
[31.375, 0.29694444444444446]
Arousal 45.125
Counter({50.0: 25, 37.5: 25, 25.0: 16, 62.5: 16, 75.0: 10, 12.5: 7, 87.5: 1})
Valence 57.875
Counter({62.5: 27, 50.0: 27, 75.0: 24, 37.5: 13, 25.0: 4, 87.5: 4, 12.5: 1})
Arousal Valence
[45.125, 57.875, 0.43558513708513713, 0.5722785270285269]
s16
Array of average Happy Sad accurecies for user s16
Counter({80.0: 40, 100.0: 35, 60.0: 21, 40.0: 3, 0.0: 1})
Happy Sad
[80.8, 0.7975952380952381]
Array of average All Emotions accurecies for user s16
Counter({37.5: 35, 50.0: 20, 62.5: 17, 25.0: 15, 12.5: 6, 75.0: 4, 87.5: 2, 0.0: 1})
Combined Emotions
[43.0, 0.38015223665223663]
Arousal 66.5
Counter({62.5: 27, 75.0: 25, 50.0: 21, 87.5: 17, 37.5: 6, 100.0: 3, 2

Array of average Happy Sad accurecies for user s29
Counter({66.66666666666666: 40, 50.0: 29, 83.33333333333334: 20, 100.0: 5, 33.33333333333333: 5, 16.666666666666664: 1})
Happy Sad
[64.66666666666667, 0.643825036075036]
Array of average All Emotions accurecies for user s29
Counter({50.0: 34, 62.5: 25, 37.5: 15, 75.0: 11, 25.0: 10, 12.5: 3, 87.5: 2})
Combined Emotions
[51.125, 0.4690764721389722]
Arousal 72.375
Counter({75.0: 37, 87.5: 22, 62.5: 20, 50.0: 12, 100.0: 6, 25.0: 2, 37.5: 1})
Valence 59.75
Counter({50.0: 34, 62.5: 30, 75.0: 20, 37.5: 10, 87.5: 6})
Arousal Valence
[72.375, 59.75, 0.6935142912642912, 0.5964177489177489]
s30
Array of average Happy Sad accurecies for user s30
Counter({57.14285714285714: 34, 42.857142857142854: 24, 71.42857142857143: 24, 85.71428571428571: 10, 28.57142857142857: 7, 14.285714285714285: 1})
Happy Sad
[57.57142857142857, 0.5564794094794095]
Array of average All Emotions accurecies for user s30
Counter({50.0: 38, 37.5: 26, 62.5: 14, 25.0: 11, 75.0: 